# MedGemma Clinical Assistant

AI-powered clinical decision support with MedGemma for the MedGemma Impact Challenge.

## Features
- 🏥 Multimodal reasoning (images + text)
- 📋 SOAP note generation
- 🔢 ICD-10 code suggestions
- 💊 Drug interaction checking
- 📊 Differential diagnosis with confidence scores

## Setup

In [ ]:
# Install dependencies
!pip install -q transformers accelerate bitsandbytes pillow fhir-resources

In [ ]:
import sys
sys.path.insert(0, '/kaggle/input/medgemma-clinical-assistant')

# Import our clinical intelligence module
from src.clinical import get_clinical_intelligence, ClinicalIntelligence
from src.soap import SOAPGenerator, EnhancedSOAPNote

## Demo: Clinical Intelligence Features

In [ ]:
# Initialize clinical intelligence
ci = get_clinical_intelligence()
print("Clinical Intelligence initialized!")

### 1. ICD-10 Code Lookup

In [ ]:
# Look up ICD-10 codes for common diagnoses
diagnoses = ["pneumonia", "COPD", "diabetes", "hypertension"]

print("ICD-10 Code Lookups:")
print("-" * 50)
for dx in diagnoses:
    code = ci.lookup_icd10(dx)
    if code:
        print(f"{dx.title():20} → {code['code']:10} {code['description']}")

### 2. Drug Interaction Checking

In [ ]:
# Check for drug interactions
current_meds = ["Lisinopril 10mg", "Aspirin 81mg", "Metformin 500mg"]
new_meds = ["Spironolactone 25mg"]

print("Drug Interaction Check")
print(f"Current: {current_meds}")
print(f"New:     {new_meds}")
print("-" * 50)

interactions = ci.check_drug_interactions(current_meds, new_meds)
if interactions:
    for i in interactions:
        print(f"⚠️ {i.drug1} + {i.drug2}")
        print(f"   Severity: {i.severity.upper()}")
        print(f"   Effect: {i.effect}")
else:
    print("✅ No significant interactions detected")

### 3. Critical Finding Detection

In [ ]:
# Detect critical findings in clinical text
imaging_report = """
Chest CT: Right lower lobe pulmonary nodule measuring 8mm.
No evidence of pulmonary embolism. 
Mild cardiomegaly noted.
"""

print("Critical Finding Detection")
print("-" * 50)

alerts = ci.detect_critical_findings(imaging_report, source="imaging")
for alert in alerts:
    emoji = "🚨" if alert.severity == "critical" else "⚠️"
    print(f"{emoji} {alert.finding} ({alert.severity.upper()})")
    print(f"   Source: {alert.source}")
    print(f"   Action: {alert.recommendation}")

### 4. Differential Diagnosis with Confidence

In [ ]:
# Generate differential diagnoses
symptoms = ["cough", "dyspnea", "wheezing", "fever"]
patient_history = {
    "conditions": [{"name": "Asthma"}, {"name": "Smoking history"}]
}
imaging = "Chest X-ray shows bilateral infiltrates"

print("Differential Diagnoses (Ranked by Confidence)")
print("-" * 50)

diffs = ci.generate_differential_with_confidence(symptoms, patient_history, imaging)
for i, d in enumerate(diffs, 1):
    print(f"{i}. {d.diagnosis}")
    print(f"   Confidence: {d.confidence_percent}")
    print(f"   ICD-10: {d.icd10_code or 'N/A'}")
    print(f"   Evidence: {', '.join(d.evidence)}")

### 5. Enhanced SOAP Note Generation

In [ ]:
# Generate enhanced SOAP note with clinical intelligence
sg = SOAPGenerator()

transcription = """
67-year-old male presents with 3 weeks of worsening cough and shortness of breath.
History of COPD, former smoker. Using rescue inhaler 4-5 times daily.
Denies fever, chills. Reports fatigue.
Vitals: BP 142/88, HR 92, SpO2 94% on room air.
Lungs: Bilateral expiratory wheezes.
"""

patient_context = {
    "patient": {"name": "John Doe", "age": 67, "gender": "male"},
    "conditions": [{"name": "COPD"}, {"name": "Hypertension"}],
    "medications": [{"name": "Albuterol"}, {"name": "Lisinopril"}]
}

enhanced_soap = sg.generate_enhanced_soap(
    transcription=transcription,
    patient_context=patient_context,
    image_findings="Chest X-ray: hyperinflation, RLL opacity"
)

print("Enhanced SOAP Note")
print("=" * 50)
print(f"Differentials: {len(enhanced_soap.differentials)}")
print(f"Drug Interactions: {len(enhanced_soap.drug_interactions)}")
print(f"Critical Alerts: {len(enhanced_soap.critical_alerts)}")
print(f"\nSubjective:\n{enhanced_soap.subjective[:200]}...")

## MedGemma Image Analysis (GPU Required)

In [ ]:
# Only run this cell if you have GPU access
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Load MedGemma (requires GPU with 8GB+ VRAM)
# Uncomment to run on Kaggle with GPU

# from src.agent import MedGemmaAgent
# agent = MedGemmaAgent(load_in_4bit=True)
# 
# # Analyze a chest X-ray
# result = agent.analyze_image(
#     image_path="/path/to/xray.jpg",
#     clinical_context="67yo male, cough and dyspnea",
#     modality="xray"
# )
# print(result["analysis"])

## Summary

This notebook demonstrates the MedGemma Clinical Assistant's key features:

1. ✅ **ICD-10 Codes** - Instant diagnosis code lookup
2. ✅ **Drug Interactions** - Safety checks for medication combinations
3. ✅ **Critical Alerts** - Urgent finding detection from clinical text
4. ✅ **Differential Diagnosis** - Ranked by confidence with evidence
5. ✅ **Enhanced SOAP Notes** - Structured documentation with clinical intelligence

For full multimodal (image) analysis, run with GPU access and load the MedGemma model.